<a href="https://colab.research.google.com/github/sagar9926/Natural-Language-Processing/blob/main/END_Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 TorchText

In [1]:
!git clone https://github.com/prrao87/fine-grained-sentiment.git

fatal: destination path 'fine-grained-sentiment' already exists and is not an empty directory.


In [2]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [3]:
!cp /content/fine-grained-sentiment/data/sst/sst_dev.txt /content/data
!cp /content/fine-grained-sentiment/data/sst/sst_test.txt /content/data
!cp /content/fine-grained-sentiment/data/sst/sst_train.txt /content/data
   

In [4]:
# Import Library
import random
import torch, torchtext
from torchtext import data
from torchtext import datasets
import pandas as pd
import numpy as np

# Manual Seed
SEED = 43
torch.manual_seed(SEED)
import spacy
spacy_en = spacy.load('en')

In [5]:
df_train = pd.read_csv('/content/data/sst_train.txt', sep='\t', header=None,
                   names=['sentiment', 'review'],
                  )
df_train['sentiment'] = df_train['sentiment'].str.replace('__label__', '')
df_train['sentiment'] = df_train['sentiment'].astype(int)

df_test = pd.read_csv('/content/data/sst_test.txt', sep='\t', header=None,
                   names=['sentiment', 'review'],
                  )
df_test['sentiment'] = df_test['sentiment'].str.replace('__label__', '')
df_test['sentiment'] = df_test['sentiment'].astype(int)


df_dev = pd.read_csv('/content/data/sst_dev.txt', sep='\t', header=None,
                   names=['sentiment', 'review'],
                  )
df_dev['sentiment'] = df_dev['sentiment'].str.replace('__label__', '')
df_dev['sentiment'] = df_dev['sentiment'].astype(int)


In [6]:
df_train.head()

,sentiment,review
0,4,The Rock is destined to be the 21st Century 's...
1,5,The gorgeously elaborate continuation of `` Th...
2,4,Singer/composer Bryan Adams contributes a slew...
3,3,You 'd think by now America would have had eno...
4,4,Yet the act is still charming here .


In [7]:
#preprocess_pipeline = data.Pipeline(lambda x: x + "!")

In [8]:
Review = data.Field(sequential = True, tokenize = 'spacy' , batch_first =True, include_lengths=True)
Sentiment = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [9]:
train_fields = [('review', Review),('sentiment',Sentiment)]
dev_fields = [('review', Review),('sentiment',Sentiment)]
test_fields = [('review', Review),('sentiment',Sentiment)]

In [10]:
train_example = [data.Example.fromlist([df_train.review[i],df_train.sentiment[i]], train_fields) for i in range(df_train.shape[0])] 
dev_example = [data.Example.fromlist([df_dev.review[i],df_dev.sentiment[i]], dev_fields) for i in range(df_dev.shape[0])] 
test_example = [data.Example.fromlist([df_test.review[i],df_test.sentiment[i]], test_fields) for i in range(df_test.shape[0])] 

In [11]:
trainDataset = data.Dataset(train_example, train_fields)
devDataset = data.Dataset(dev_example, dev_fields)
testDataset = data.Dataset(test_example, test_fields)

In [12]:
vars(trainDataset.examples[0])

{'review': ['The',
  'Rock',
  'is',
  'destined',
  'to',
  'be',
  'the',
  '21st',
  'Century',
  "'s",
  'new',
  '`',
  '`',
  'Conan',
  "''",
  'and',
  'that',
  'he',
  "'s",
  'going',
  'to',
  'make',
  'a',
  'splash',
  'even',
  'greater',
  'than',
  'Arnold',
  'Schwarzenegger',
  ',',
  'Jean',
  '-',
  'Claud',
  'Van',
  'Damme',
  'or',
  'Steven',
  'Segal',
  '.'],
 'sentiment': 4}

### Augmentation

In [13]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [14]:
random_deletion([1,2,3,4,5,6,7])

[4, 5, 6]

In [15]:
def random_swap(words, n=5): 
    length = range(len(words)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        words[idx1], words[idx2] = words[idx2], words[idx1] 
    return words

In [16]:
random_swap(["My","Name","Is","Sagar"])

['Is', 'Sagar', 'Name', 'My']

In [17]:
!pip install google_trans_new

In [18]:
import random
import google_trans_new
from google_trans_new import google_translator

def back_trans(words):
  translator = google_translator()
  sentence = " ".join(words)

  available_langs = list(google_trans_new.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  #print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

  translations = translator.translate(sentence, lang_tgt=trans_lang) 
  
  translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt='en') 
  
  return translations_en_random.split(" ")

In [19]:
back_trans(["The" , "dog" , "is" , "sleeping", "on","mat"])

['The', 'dog', 'is', 'sleeping', 'on', 'a', 'carpet', '']

In [20]:
def augmentation() :

  ## we will apply augmentation only 50% of times

  aug_funcs = [random_swap , back_trans,random_deletion]
  p = np.random.uniform()
  
  if 0 < p < 0.5 :
    func_index =  list(set([np.random.randint(0,3) for _ in range(3)]))
    augment = [aug_funcs[i] for i in func_index]
  else : 
    augment = []
  return augment

In [21]:
Review.build_vocab(trainDataset)
Sentiment.build_vocab(trainDataset)

In [22]:
print('Size of input vocab : ', len(Review.vocab))
print('Size of label vocab : ', len(Sentiment.vocab))
print('Top 10 words appreared repeatedly :', list(Review.vocab.freqs.most_common(10)))
print('Labels : ', Sentiment.vocab.stoi)

Size of input vocab :  17165
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 8041), (',', 7131), ('the', 6087), ('and', 4474), ('of', 4446), ('a', 4423), ('to', 3024), ('-', 2739), ("'s", 2544), ('is', 2540)]
Labels :  defaultdict(<function _default_unk_index at 0x7f71b8b200d0>, {4: 0, 2: 1, 3: 2, 5: 3, 1: 4})


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
train_iterator, valid_iterator = data.BucketIterator.splits((trainDataset, devDataset), batch_size = 32, 
                                                            sort_key = lambda x: len(x.review),
                                                            sort_within_batch=True, device = device)

In [25]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Review.vocab.stoi, tokens)

In [26]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [27]:
# Define hyperparameters
size_of_vocab = len(Review.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [28]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(17165, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 5,391,605 trainable parameters


In [29]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [30]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  

    ## applying augmentation
    for i in range(len(train_iterator.dataset.examples)):
      aug = augmentation()
      if len(train_iterator.dataset.examples[i].review) > 10 :
        if bool(aug):
          for func in aug : 
            #print("before",train_iterator.dataset.examples[i].review,len(train_iterator.dataset.examples[i].review))
            train_iterator.dataset.examples[i].review = func(train_iterator.dataset.examples[i].review) 
            #print("after",train_iterator.dataset.examples[i].review)

    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.review   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()
        
        # compute the loss
        loss = criterion(predictions, batch.sentiment)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.sentiment)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [31]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.review
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.sentiment)
            acc = binary_accuracy(predictions, batch.sentiment)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

In [ ]:
len(train_iterator.dataset.examples)

In [ ]:
len(train_iterator)

In [ ]:
train_iterator.dataset.examples[8].review

In [ ]:
bool([1])